In [3]:
#!/usr/bin/env python3
"""
🎯 Script de Normalização do Dataset HuMob - VERSÃO CORRIGIDA
===========================================================

Normaliza os dados do HuMob e salva um dataset já pré-processado.
Isso resolve problemas de explosão de gradientes ao colocar todas as 
features na mesma escala.

Uso:
    python normalize_humob_data_fixed.py
"""

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
from pathlib import Path
from tqdm import tqdm
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings('ignore')

class HuMobNormalizer:
    """
    Normaliza dataset HuMob com estratégias específicas para cada tipo de feature.
    
    🎯 Analogia: Como calibrar uma receita onde cada ingrediente tem escala diferente
    - Coordenadas (x,y): Como quilos de farinha → normalizar para [0,1]
    - Tempo (d,t): Como colheres de sal → normalizar ciclicamente  
    - POIs: Como gotas de corante → log-transform + normalize
    """
    
    def __init__(self, input_path: str, output_path: str):
        self.input_path = Path(input_path)
        self.output_path = Path(output_path)
        
        # Scalers que serão salvos para uso posterior
        self.coords_scaler = MinMaxScaler(feature_range=(0, 1))
        self.poi_stats = {}  # Para log1p + normalização manual
        
        # Metadados para salvar junto
        self.normalization_info = {
            "version": "1.0",
            "strategy": {
                "coordinates": "MinMaxScaler [0,1]",
                "days": "Linear [0,1]", 
                "timeslots": "Circular sin/cos",
                "pois": "log1p + per-column normalization",
                "cities": "Label encoding A=0, B=1, C=2, D=3"
            }
        }
    
    def compute_statistics_robust(self, chunk_size: int = 50_000):
        """
        Calcula estatísticas com tratamento robusto de POIs problemáticos.
        PROCESSA TODOS OS CHUNKS, não só os primeiros 5!
        """
        print("📊 Calculando estatísticas dos dados...")
        
        # Acumuladores para coordenadas
        all_coords = []
        
        # Acumuladores para POIs (por coluna) - COM LIMPEZA
        poi_sums = np.zeros(85)
        poi_counts = 0
        poi_max_vals = np.zeros(85)
        
        # Ranges de tempo
        max_day = 0
        
        # Contadores para debug
        problematic_chunks = 0
        total_problematic_pois = 0
        
        pf = pq.ParquetFile(self.input_path)
        total_batches = (pf.metadata.num_rows + chunk_size - 1) // chunk_size
        
        print(f"📁 Processando {pf.metadata.num_rows:,} linhas em {total_batches} chunks...")
        
        for i, batch in enumerate(tqdm(pf.iter_batches(batch_size=chunk_size), 
                                      total=total_batches, desc="Calculando stats")):
            
            table = pa.Table.from_batches([batch], schema=pf.schema_arrow)
            
            # 1. COORDENADAS (como sempre)
            x_vals = table.column("x").to_numpy()
            y_vals = table.column("y").to_numpy()
            coords = np.column_stack([x_vals, y_vals])
            all_coords.append(coords)
            
            # 2. DIAS (como sempre)
            d_vals = table.column("d").to_numpy()
            max_day = max(max_day, d_vals.max())
            
            # 3. POIs COM LIMPEZA ROBUSTA
            try:
                poi_lists = table.column("POI").to_pylist()
                
                # Limpa POIs problemáticos
                cleaned_poi_lists = []
                chunk_problems = 0
                
                for poi_list in poi_lists:
                    if poi_list is None:
                        cleaned_poi_lists.append([0.0] * 85)
                        chunk_problems += 1
                    elif not isinstance(poi_list, (list, tuple)):
                        cleaned_poi_lists.append([0.0] * 85)
                        chunk_problems += 1
                    elif len(poi_list) != 85:
                        # Ajusta tamanho
                        if len(poi_list) < 85:
                            poi_fixed = list(poi_list) + [0.0] * (85 - len(poi_list))
                        else:
                            poi_fixed = list(poi_list)[:85]
                        
                        # Converte valores para float seguro
                        poi_cleaned = []
                        for val in poi_fixed:
                            try:
                                float_val = float(val) if val is not None else 0.0
                                if not np.isfinite(float_val) or float_val < 0:
                                    float_val = 0.0
                                poi_cleaned.append(float_val)
                            except (ValueError, TypeError):
                                poi_cleaned.append(0.0)
                        
                        cleaned_poi_lists.append(poi_cleaned)
                        chunk_problems += 1
                    else:
                        # Converte valores para float seguro
                        poi_cleaned = []
                        for val in poi_list:
                            try:
                                float_val = float(val) if val is not None else 0.0
                                if not np.isfinite(float_val) or float_val < 0:
                                    float_val = 0.0
                                poi_cleaned.append(float_val)
                            except (ValueError, TypeError):
                                poi_cleaned.append(0.0)
                        
                        cleaned_poi_lists.append(poi_cleaned)
                
                # Converte para array e processa
                poi_array = np.array(cleaned_poi_lists, dtype=np.float32)
                
                # Limpa qualquer NaN/Inf restante
                poi_array = np.nan_to_num(poi_array, nan=0.0, posinf=0.0, neginf=0.0)
                
                # Acumula estatísticas
                poi_sums += poi_array.sum(axis=0)
                poi_counts += poi_array.shape[0]
                poi_max_vals = np.maximum(poi_max_vals, poi_array.max(axis=0))
                
                # Conta problemas
                if chunk_problems > 0:
                    problematic_chunks += 1
                    total_problematic_pois += chunk_problems
                
            except Exception as e:
                print(f"⚠️ Erro no chunk {i}: {str(e)}")
                problematic_chunks += 1
                # Usa zeros para este chunk problemático
                poi_array = np.zeros((chunk_size, 85), dtype=np.float32)
                poi_sums += poi_array.sum(axis=0)
                poi_counts += min(chunk_size, len(x_vals))  # Usa tamanho real do chunk
        
        # Calcula scalers das coordenadas
        all_coords = np.vstack(all_coords)
        self.coords_scaler.fit(all_coords)
        
        # Salva estatísticas POI
        self.poi_stats = {
            'max_vals': poi_max_vals.tolist(),
            'mean_vals': (poi_sums / max(poi_counts, 1)).tolist()  # Evita divisão por zero
        }
        
        # Salva info de tempo
        self.max_day = max_day
        
        print(f"✅ Estatísticas calculadas:")
        print(f"   📍 Coordenadas: x[{all_coords[:,0].min():.0f}, {all_coords[:,0].max():.0f}], y[{all_coords[:,1].min():.0f}, {all_coords[:,1].max():.0f}] → [0,1]")
        print(f"   📅 Dias: [0, {max_day}]")
        print(f"   🏢 POIs: max_sum={poi_max_vals.max():.0f}, mean_sum={poi_sums.max()/max(poi_counts,1):.2f}")
        
        if problematic_chunks > 0 or total_problematic_pois > 0:
            print(f"   🔧 Limpeza POI: {problematic_chunks} chunks com problemas, {total_problematic_pois:,} POIs corrigidos")
        else:
            print(f"   ✅ POIs: Nenhum problema encontrado!")
    
    def normalize_coordinates(self, x_vals, y_vals):
        """Normaliza coordenadas para [0, 1]"""
        coords = np.column_stack([x_vals, y_vals])
        coords_norm = self.coords_scaler.transform(coords)
        return coords_norm[:, 0], coords_norm[:, 1]
    
    def normalize_time(self, d_vals, t_vals):
        """
        Normaliza tempo:
        - Dias: linear [0,1]  
        - Timeslots: circular (sin/cos)
        """
        # Dias: normalização linear
        d_norm = d_vals.astype(np.float32) / self.max_day
        
        # Timeslots: normalização circular (importante para horários!)
        t_sin = np.sin(2 * np.pi * t_vals / 48).astype(np.float32)
        t_cos = np.cos(2 * np.pi * t_vals / 48).astype(np.float32)
        
        return d_norm, t_sin, t_cos
    
    def normalize_pois_robust(self, poi_lists):
        """
        Normaliza POIs com estratégia robusta - VERSÃO MELHORADA
        """
        # Limpeza preventiva
        valid_poi_lists = []
        
        for poi_list in poi_lists:
            # Trata casos problemáticos
            if poi_list is None or not isinstance(poi_list, (list, tuple)):
                valid_poi_lists.append([0.0] * 85)
                continue
            
            # Converte e ajusta tamanho
            if len(poi_list) < 85:
                poi_fixed = list(poi_list) + [0.0] * (85 - len(poi_list))
            elif len(poi_list) > 85:
                poi_fixed = list(poi_list)[:85]
            else:
                poi_fixed = list(poi_list)
            
            # Converte para float e limpa
            poi_cleaned = []
            for val in poi_fixed:
                try:
                    float_val = float(val) if val is not None else 0.0
                    if not np.isfinite(float_val):
                        float_val = 0.0
                    poi_cleaned.append(max(0.0, float_val))  # Remove negativos também
                except (ValueError, TypeError):
                    poi_cleaned.append(0.0)
            
            valid_poi_lists.append(poi_cleaned)
        
        # Converte para array
        poi_array = np.array(valid_poi_lists, dtype=np.float32)
        
        # Aplica log1p e normalização como antes
        poi_log = np.log1p(poi_array)
        
        max_vals = np.array(self.poi_stats['max_vals'])
        max_vals_log = np.log1p(max_vals)
        max_vals_log[max_vals_log == 0] = 1.0
        
        poi_normalized = poi_log / max_vals_log[np.newaxis, :]
        poi_normalized = np.clip(poi_normalized, 0.0, 1.0)
        
        return poi_normalized.tolist()
    
    def normalize_cities(self, city_vals):
        """Converte cidades para encoding numérico"""
        city_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
        return np.array([city_mapping[c] for c in city_vals], dtype=np.int8)
    
    def process_and_save(self, chunk_size: int = 50_000):
        """
        Processa e salva dataset normalizado chunk por chunk.
        """
        print("🔄 Normalizando e salvando dados...")
        
        pf = pq.ParquetFile(self.input_path)
        total_batches = (pf.metadata.num_rows + chunk_size - 1) // chunk_size
        
        writer = None
        processed_rows = 0
        
        for batch in tqdm(pf.iter_batches(batch_size=chunk_size), 
                         total=total_batches, desc="Normalizando"):
            
            table = pa.Table.from_batches([batch], schema=pf.schema_arrow)
            
            # Extrai dados originais
            uid_vals = table.column("uid").to_numpy()
            d_vals = table.column("d").to_numpy() 
            t_vals = table.column("t").to_numpy()
            x_vals = table.column("x").to_numpy()
            y_vals = table.column("y").to_numpy()
            city_vals = table.column("city").to_pylist()
            poi_lists = table.column("POI").to_pylist()
            
            # Aplica normalizações
            x_norm, y_norm = self.normalize_coordinates(x_vals, y_vals)
            d_norm, t_sin, t_cos = self.normalize_time(d_vals, t_vals)
            poi_norm = self.normalize_pois_robust(poi_lists)
            city_encoded = self.normalize_cities(city_vals)
            
            # Cria novo schema
            if writer is None:
                new_schema = pa.schema([
                    pa.field("uid", pa.int64()),
                    pa.field("d_orig", pa.uint8()),  # Original para referência  
                    pa.field("t_orig", pa.uint8()),  # Original para referência
                    pa.field("d_norm", pa.float32()),  # Normalizado [0,1]
                    pa.field("t_sin", pa.float32()),   # Circular sin
                    pa.field("t_cos", pa.float32()),   # Circular cos  
                    pa.field("x_norm", pa.float32()),  # Normalizado [0,1]
                    pa.field("y_norm", pa.float32()),  # Normalizado [0,1]
                    pa.field("city", pa.string()),     # Original
                    pa.field("city_encoded", pa.int8()),  # Encoded
                    pa.field("POI_norm", pa.list_(pa.float32()))  # Normalizado
                ])
                
                # Adiciona metadados
                metadata = {
                    b"normalization_info": json.dumps(self.normalization_info).encode("utf-8"),
                    b"coords_scaler_min": json.dumps(self.coords_scaler.data_min_.tolist()).encode("utf-8"),
                    b"coords_scaler_scale": json.dumps(self.coords_scaler.scale_.tolist()).encode("utf-8"),
                    b"poi_stats": json.dumps(self.poi_stats).encode("utf-8"),
                    b"max_day": str(self.max_day).encode("utf-8")
                }
                new_schema = new_schema.with_metadata(metadata)
                
                writer = pq.ParquetWriter(self.output_path, new_schema, compression='snappy')
            
            # Cria nova tabela
            new_table = pa.table({
                "uid": uid_vals,
                "d_orig": d_vals,
                "t_orig": t_vals, 
                "d_norm": d_norm,
                "t_sin": t_sin,
                "t_cos": t_cos,
                "x_norm": x_norm,
                "y_norm": y_norm,
                "city": city_vals,
                "city_encoded": city_encoded,
                "POI_norm": poi_norm
            }, schema=new_schema)
            
            writer.write_table(new_table)
            processed_rows += len(uid_vals)
        
        if writer:
            writer.close()
            
        print(f"✅ Dataset normalizado salvo em: {self.output_path}")
        print(f"📊 Linhas processadas: {processed_rows:,}")
        print(f"💾 Tamanho do arquivo: {self.output_path.stat().st_size / (1024**2):.1f} MB")
    
    def run(self, chunk_size: int = 50_000):
        """Executa normalização completa"""
        print("🎯 Iniciando normalização do dataset HuMob")
        print(f"📂 Input: {self.input_path}")
        print(f"📁 Output: {self.output_path}")
        print("="*60)
        
        # Passo 1: Calcular estatísticas (TODAS, não só debug)
        self.compute_statistics_robust(chunk_size)
        
        # Passo 2: Processar e salvar
        self.process_and_save(chunk_size)
        
        print("\n🎉 Normalização concluída com sucesso!")
        print("\n📋 Próximos passos:")
        print("   1. Teste o novo dataset com check_normalized_data()")
        print("   2. Atualize seu clnn.ipynb para usar dados normalizados")
        print("   3. Remova normalizações inline do código de treino")


def check_normalized_data(file_path: str, n_samples: int = 1000):
    """
    Verifica se os dados foram normalizados corretamente.
    """
    print(f"🔍 Verificando dados normalizados em: {file_path}")
    
    # Lê amostra
    df = pq.read_table(file_path).slice(0, n_samples).to_pandas()
    
    print(f"\n📊 Estatísticas da amostra ({len(df):,} linhas):")
    print(f"   📍 Coordenadas:")
    print(f"      x_norm: [{df['x_norm'].min():.3f}, {df['x_norm'].max():.3f}] (deve ser [0,1])")
    print(f"      y_norm: [{df['y_norm'].min():.3f}, {df['y_norm'].max():.3f}] (deve ser [0,1])")
    
    print(f"   📅 Tempo:")
    print(f"      d_norm: [{df['d_norm'].min():.3f}, {df['d_norm'].max():.3f}] (deve ser [0,1])")
    print(f"      t_sin: [{df['t_sin'].min():.3f}, {df['t_sin'].max():.3f}] (deve ser [-1,1])")
    print(f"      t_cos: [{df['t_cos'].min():.3f}, {df['t_cos'].max():.3f}] (deve ser [-1,1])")
    
    print(f"   🏢 POIs:")
    poi_sample = np.array(df['POI_norm'].iloc[0])
    print(f"      Exemplo: [{poi_sample.min():.3f}, {poi_sample.max():.3f}] (deve ser [0,1])")
    print(f"      Dimensão: {len(poi_sample)} (deve ser 85)")
    
    print(f"   🏙️ Cidades:")
    print(f"      Encoded: {sorted(df['city_encoded'].unique())} (deve ser [0,1,2,3])")
    print(f"      Original: {sorted(df['city'].unique())}")
    
    # Verifica metadados
    pf = pq.ParquetFile(file_path)
    metadata = pf.schema_arrow.metadata
    if b"normalization_info" in metadata:
        norm_info = json.loads(metadata[b"normalization_info"])
        print(f"\n✅ Metadados de normalização encontrados:")
        for key, value in norm_info["strategy"].items():
            print(f"      {key}: {value}")
    else:
        print("\n⚠️ Metadados de normalização não encontrados!")


if __name__ == "__main__":
    # Configurações
    INPUT_FILE = "humob_all_cities_dpsk.parquet"  # Seu arquivo original
    OUTPUT_FILE = "humob_all_cities_normalized.parquet"  # Arquivo normalizado
    
    # Verifica se arquivo de entrada existe
    if not Path(INPUT_FILE).exists():
        print(f"❌ Arquivo não encontrado: {INPUT_FILE}")
        print("📝 Certifique-se de que o arquivo foi gerado pelo data_loader.ipynb")
        exit(1)
    
    # Executa normalização
    normalizer = HuMobNormalizer(INPUT_FILE, OUTPUT_FILE)
    normalizer.run(chunk_size=50_000)
    
    # Verifica resultado
    print("\n" + "="*60)
    check_normalized_data(OUTPUT_FILE, n_samples=5000)
    
    print(f"\n🎯 PRONTO! Agora você pode:")
    print(f"   1. Usar '{OUTPUT_FILE}' no seu clnn.ipynb")
    print(f"   2. Remover todas as normalizações inline do código")
    print(f"   3. Usar diretamente as colunas *_norm nas features")

🎯 Iniciando normalização do dataset HuMob
📂 Input: humob_all_cities_dpsk.parquet
📁 Output: humob_all_cities_normalized.parquet
📊 Calculando estatísticas dos dados...
📁 Processando 162,785,736 linhas em 3256 chunks...


Calculando stats:   0%|          | 13/3256 [01:18<5:24:18,  6.00s/it]


KeyboardInterrupt: 